In [1]:
import pylab
import h5py
import math
import array
from numpy import *
import numpy as np
from pycbc.types import TimeSeries, FrequencySeries
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc import types, fft, waveform
import lal
from scipy import interpolate
from scipy.interpolate import interp1d
from lal import MSUN_SI, MTSUN_SI, G_SI, PC_SI, C_SI, PI
from pycbc.filter import match
from pycbc.psd import aLIGOZeroDetHighPower

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
#plt.rcParams["font.family"] = "monospace"
 
from matplotlib import gridspec

import matplotlib.pyplot as plt


lal.MSUN_SI != Msun
setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work


### Generate numerical estimates for t_amtch Amplitude, and t_shift

In [2]:
#hfile = ['1167hyb_EccTD_xlow045_GM.h5']
hfile = ['1364_EccTD_22_hyb_poster.h5']
e = [0.172]
Mtot = 30
q = 2

In [3]:
t_match_Hyb = []
t_match_byM = []
tshift_Hyb = []
tshift_Hyb_byM = []
match_Hyb = []
Idx_hyb = []
eta = []

In [4]:
for size in range(0,len(hfile)):
    hyb_name = hfile[size][0:4]
    #f = h5py.File('/home/pratul/Downloads/Project/New hybrids/1364_EccTD_22_hyb_poster.h5','r')
    #f = h5py.File('/home/pratul/Downloads/Project/hybrids/1364hyb_EccTD_xlow045_GM.h5','r')
    #f = h5py.File('/home/pratul/Downloads/Project/New hybrids/Ebersold/1364_EccTD_Ebersold_HM_hyb.h5','r')
    f = h5py.File('/home/pratul/Downloads/Project/New hybrids/TT2_Hybs/1364_TT2_Ebersold_22_hyb.h5','r')
    d = f['l2_m2']
    #d = f['OutermostExtraction.dir']['Y_l2_m2.dat']
    #x1 = np.real(d[:,0])
    #plotband=np.where(x1>=170)
    tVec = real(d[:,0])
    h22pVec = real(d[0:,1])
    h22cVec = real(d[0:,2])
    
    #tVec = real(d[:,0])
    #h22pVec = real(d[0:,1])
    #h22cVec = real(d[0:,2])
    
    mode2polfac=4*(5/(64*np.pi))**(1/2)
    hpVec=h22pVec
    hcVec=h22cVec
    
    tVec = tVec - tVec[np.argmax(abs(hpVec + 1j*hcVec))]
    

    M=Mtot
    M1 = M*q/(1+q)
    M2 = M*1/(1+q)
    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    tVec=tVec*M*MTSUN_SI
    hpVec=hpVec*G_SI*M_SI/D_SI/C_SI/C_SI * mode2polfac
    hcVec=hcVec*G_SI*M_SI/D_SI/C_SI/C_SI * mode2polfac
    
    #q.append(M1/M2)
    eta.append(M1*M2/(M*M))
    
    delta_t = delta_t_NR = np.abs(np.mean((np.diff(tVec))))
    
    
    fref = 0.075**(3/2) /MTSUN_SI/ PI/M
    fmin = 16.68 #0.045**(3/2) /MTSUN_SI/ PI/M
    
    hp, hc = get_td_waveform(approximant='EccentricTD', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin, eccentricity=e[size])
    sp, sc = get_td_waveform(approximant='SEOBNRv4', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin)
    
    tmin = max(tVec[0],hp.sample_times[0],sp.sample_times[0]) + 0.04
    
    hpVec_intrp = interp1d(tVec, hpVec, kind='cubic')
    hcVec_intrp = interp1d(tVec, hcVec, kind='cubic')
    tVec_intrp=np.arange(tmin, np.max(tVec), delta_t)
    
    hpVec_intrp = hpVec_intrp(tVec_intrp)
    hcVec_intrp = hcVec_intrp(tVec_intrp)
    tVec=tVec_intrp
    hpVec=hpVec_intrp
    hcVec=hcVec_intrp
    h22Hyb = hpVec + 1j*hcVec
    phaseHyb = -np.unwrap(np.angle(h22Hyb)*2)/2
    tVec = tVec - tVec[np.argmax(abs(h22Hyb))]
    
    #IMR_circular
    sp_intrp = interp1d(sp.sample_times, sp, kind='cubic')
    sc_intrp = interp1d(sc.sample_times, sc, kind='cubic')
    tImr_intrp=np.arange(tmin, sp.sample_times[-1], delta_t)
    sp_intrp = sp_intrp(tImr_intrp)
    sc_intrp = sc_intrp(tImr_intrp)
    tImr = tImr_intrp
    hpImr = sp_intrp
    hcImr = sc_intrp
    h22Imr = hpImr + 1j*hcImr

    
    htarg = TimeSeries(h22Hyb,delta_t,epoch=tVec[0]-tVec[np.argmax(abs(h22Hyb))])
    MatchMax = []
    IdxMax = []
    t_shift = []
    timeS = []
    xJoinMax = []
    
    #find match
    tshift = -400*M*MTSUN_SI
    while(True):
        if tshift>400*M*MTSUN_SI: 
            break

        tMatchIdx = []
        MatchVal = []
        timeJoin = []
        #xJoin = []
        # interpolation EccentricTD
        hp_intrp = interp1d(hp.sample_times-tshift, hp, kind='cubic',fill_value='extrapolate')
        hc_intrp = interp1d(hc.sample_times-tshift, hc, kind='cubic',fill_value='extrapolate')
        # new (unifrom) sampling rate (for interpolation and further processing)
        tEcc_intrp=np.arange(tmin, hp.sample_times[-1]-tshift, delta_t)
        hp_intrp = hp_intrp(tEcc_intrp)
        hc_intrp = hc_intrp(tEcc_intrp)
        tEcc = tEcc_intrp
        hpEcc = hp_intrp
        hcEcc = hc_intrp
        h22Ecc = hpEcc + 1j*hcEcc
        #print(len(h22Ecc))
        phaseEcc = np.unwrap(np.angle(h22Ecc))
        phaseImr = np.unwrap(np.angle(h22Imr))


        ti = -450
        while (ti<=-30):
            
            if ti>=tEcc[-1]:
                break
                
            Idxjoin = np.argmin(abs(tEcc-(ti*M*MTSUN_SI)))

            htarg = TimeSeries(h22Hyb,delta_t,epoch=tVec[0]-tVec[np.argmax(abs(h22Hyb))])

            #setting phase at joining point to be equal
            psiEcc = phaseEcc[Idxjoin]
            psiImr = phaseImr[Idxjoin]
            dpsi = psiImr - psiEcc                                          
            hp_new = real(h22Ecc)*np.cos(dpsi) - imag(h22Ecc)*np.sin(dpsi)
            hc_new = real(h22Ecc)*np.sin(dpsi) + imag(h22Ecc)*np.cos(dpsi)
            h22Ecc_new = hp_new + 1j*hc_new


            "alternative model"
            idxstr = np.argmin(abs(tEcc-((ti-500)*M*MTSUN_SI)))
            #idxstr = Idxjoin - 100
            amp = []
            count = 0
            length = Idxjoin - idxstr 
            for i in range(idxstr,Idxjoin):
                amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
                count = count+1

            t_model = np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))

            h22amp = np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
            h22amp_model = np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))
            

            htemp = TimeSeries(h22amp_model,delta_t,epoch=t_model[0]-t_model[np.argmax(abs(h22amp_model))])
            tlen = max(len(htemp),len(htarg))
            htarg.resize(tlen)
            htemp.resize(tlen)
            phase_targ = np.unwrap(np.angle(htarg)*2)/2

            h22p_model = abs(htemp) * np.cos(phase_targ)
            hp_temp = TimeSeries(h22p_model,delta_t,epoch=t_model[0]-t_model[np.argmax(abs(h22amp_model))])
            hp_targ = htarg.real()

            tlen=max(len(hp_temp),len(hp_targ))
            hp_temp.resize(tlen)
            hp_targ.resize(tlen)
            # Generate the aLIGO ZDHP PSD
            delta_f = 1.0 / htarg.duration
            flen = tlen//2 + 1
            f_low = fmin
            psd = aLIGOZeroDetHighPower(flen, delta_f, f_low)
            # Note: This takes a while the first time as an FFT plan is generated
            # subsequent calls are much faster.
            m, i = match(hp_targ, hp_temp, psd=psd, low_frequency_cutoff=f_low)

            MatchVal.append(m)
            tMatchIdx.append(Idxjoin)
            timeJoin.append(tEcc[Idxjoin])

            ti = ti+5

        j = np.argmax(MatchVal)
        MatchMax.append(MatchVal[j])
        IdxMax.append(tMatchIdx[j])
        #print(IdxMax)
        t_shift.append(tshift)
        timeS.append(timeJoin[j])

        
        tshift = tshift+(5*M*MTSUN_SI)
        
        #break
        
    maxidx = np.argmax(MatchMax)
    Idx_hyb.append(IdxMax[maxidx])
    tshift_Hyb.append(t_shift[maxidx])
    tshift_Hyb_byM.append(t_shift[maxidx]/(M*MTSUN_SI))
    match_Hyb.append(MatchMax[maxidx])
    t_match_Hyb.append(timeS[maxidx])
    t_match_byM.append(timeS[maxidx]/(M*MTSUN_SI))
    #x_hyb.append(xJoinMax[maxidx])

In [5]:
print(match_Hyb,t_match_byM, tshift_Hyb_byM)

[0.9977236392301344] [-450.2451515296988] [-344.99999999999994]
